In [5]:
import numpy as np
import copy
import random
from scipy.spatial.distance import cityblock
%run a_star_2.ipynb
from matplotlib import cm

In [6]:
maze_size

0

In [7]:
def display2(matrix):
    """
    This function is used to display maze with traversed path(highlighted)
    trajectory_path: path traversed by the agent so far
    matrix: maze of the problem
    """
    temp = matrix.copy()
    colormap = cm.get_cmap("Greys")
    plt.imshow(temp, cmap=colormap)
    plt.show()

In [8]:
def run_agent_6(agent6, maze):
#     print("Agent6 started.\nMaze Size: ",maze_size," Source: ",maze.source," Target: ",maze.target)
    global maze_size
    source = maze.source
    terrain_matrix = maze.terrain_matrix
    full_grid_world_matrix = maze.full_grid_world_matrix
    agent_gridworld = agent6.agent_gridworld
    
    target_reached = False
    while not target_reached:
        current_target = agent6.getMaxProbabilityCell(source)
#         print('source:', source, "current_target: ",current_target)
        
#             print("astar args: source:",source,"target:",current_target,"gridworld:",agent6.agent_gridworld_0_1 )
        initialize(maze_size,0.3)
        path = astar(source,current_target, agent6.agent_gridworld_0_1)
#         print(path)\
        if(len(path)==0):
            agent6.agent_gridworld_0_1[current_target[0],current_target[1]] = 1
            agent6.update_cells_when_xy_blocked(current_target[0], current_target[1], maze_size)
            agent6.agent_gridworld.itemset(current_target,3)
            maze.full_grid_world_matrix[current_target[0],current_target[1]] =1
            maze.terrain_matrix[current_target[0],current_target[1]] = 3
            continue
        
        for i in range(len(path)):
            node = path[i]
            agent6.actions += 1
#             print("belief matrix sum: ",agent6.belief_matrix.sum())
            if full_grid_world_matrix.item(node) == 0:
                terrain_type = terrain_matrix.item(node)
                agent6.agent_gridworld.itemset(node, terrain_type)
                if i == len(path)-1:
                    agent6.actions += 1
                    if(terrain_type == 0):
                        agent6.flat_exam += 1
                    elif(terrain_type == 1):
                        agent6.hilly_exam += 1
                    else:
                        agent6.forest_exam += 1
                    if(agent6.examine(node, terrain_type, maze)):
#                         print("found goal")
                        target_reached = True
                        break
                    else:
                        #update probabiities when failed to be found and terrain type=terrain_matrix.item(node)
                        agent6.update_cells_when_xy_failed_with_some_terrainType(node[0], node[1], terrain_type, maze_size)
                        source = path[i]
#                         print(agent6.belief_matrix)
                        break
            elif full_grid_world_matrix.item(node) == 1:
                agent6.actions += 2
#                 print(node,'<-cell blocked, replanning route')
                #agent6.update_probabilities(node,'blocked')
                agent6.agent_gridworld_0_1[node[0],node[1]] = 1
#                 print("In node:",node,"\nbefore: ",agent6.belief_matrix)
                agent6.update_cells_when_xy_blocked(node[0], node[1], maze_size)
                agent6.agent_gridworld.itemset(node,3)
                source = path[i-1]
#                 print("after:",agent6.belief_matrix)
                break


In [9]:
class agent_6:

    def __init__(self, maze_size):
        self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
        self.visited_matrix = np.matrix(np.zeros((maze_size, maze_size),dtype=bool))
        self.agent_gridworld = np.matrix(np.zeros((maze_size, maze_size)))
        self.agent_gridworld_0_1 = np.matrix(np.zeros((maze_size, maze_size)))
        self.actions = 0
        self.flat_exam = 0
        self.hilly_exam = 0
        self.forest_exam = 0

#     def display_node(self):
#         print("Node: ", self.row, "," , self.column)
#         print("\n the number of neighbors cell x has: ", self.Nx)
#         print("visited  Whether or not cell x has been visited: ", self.visited)
#         print("blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed: ", self.blocked)
#         print("The undiscovered neighbors are: ", self.neighbor_list)
#         print("The confirmed neighbors that are free: ", self.free_neighbors)
#         print("The confirmed neighbors that are blocked: ", self.blocked_neighbors)

        
        
    def getMaxProbabilityCell(self, source):
        mat = self.belief_matrix
        
        maxProb = mat.max()
        occurences = list(np.where(mat == maxProb))
        number_of_occurrences = len(occurences[0])
        node_to_return = source
        if number_of_occurrences > 1:
            dist = []
            previous_manhattan_distance = 9999
            for index in range(number_of_occurrences):
                
                _row = occurences[0][index]
                _col = occurences[1][index]
                
                target_node = (_row, _col)
                # find manhattan distance
                current_manhattan_distance = cityblock([source], target_node)
                dist.append([target_node,current_manhattan_distance])

                # code for finding manhattan distance between current node and the node in loop
                if current_manhattan_distance < previous_manhattan_distance:
                    previous_manhattan_distance = current_manhattan_distance
#                     node_to_return = target_node
            dist = [x for x in dist if x[1]==previous_manhattan_distance]
            node_to_return = dist[np.random.randint(0,len(dist))][0]
        
        else:
            row = occurences[0][0]
            col = occurences[1][0]
            node_to_return = (row, col)
        
        
        return node_to_return
    
    
    def examine(self, node, terrain_type, maze):
        if node ==  maze.target:
#             print("examine target")
            p = np.random.rand()
            if terrain_type == 0:
                if p >= 0.2:
                    return True
            elif terrain_type == 1:
                if p >= 0.5:
                    return True
            elif terrain_type == 2:
                if p >= 0.8:
                    return True
        
        return False
        
    def update_cells_when_xy_blocked(self, x, y, maze_size):
        denominator = 1 - self.belief_matrix[x][y]
        self.belief_matrix = self.belief_matrix/denominator
        
        self.belief_matrix[x][y]=0
        
        
#         print(self.belief_matrix)
    
    
    def update_cells_when_xy_failed_with_some_terrainType(self, x, y, t, maze_size):
        
        if t == 0:
            p_xy = 0.2*self.belief_matrix[x][y]
        elif t == 1:
            p_xy= 0.5*self.belief_matrix[x][y]
        elif t == 2:
            p_xy= 0.8*self.belief_matrix[x][y]
        
        self.belief_matrix[x][y] = p_xy
        self.belief_matrix = self.belief_matrix/self.belief_matrix.sum()
        

In [10]:
def run_agent_7(agent7, maze):
    
    global maze_size
    
    source = maze.source
    terrain_matrix = maze.terrain_matrix
    full_grid_world_matrix = maze.full_grid_world_matrix
    agent_gridworld = agent7.agent_gridworld

    target_reached = False
    while not target_reached:
        current_target = agent7.getMaxCertaintyCell(source)
#         print('source:', source, "current_target: ",current_target)
        
#             print("astar args: source:",source,"target:",current_target,"gridworld:",agent7.agent_gridworld_0_1 )
        initialize(maze_size,0.3)
        path = astar(source,current_target, agent7.agent_gridworld_0_1)
#         print(path)\
        if(len(path)==0):
            agent7.agent_gridworld_0_1[current_target[0],current_target[1]] = 1
            agent7.update_beliefs_when_xy_blocked(current_target[0], current_target[1], maze_size)
            agent7.update_certainties(maze)
            
            agent7.agent_gridworld.itemset(current_target,3)
            maze.full_grid_world_matrix[current_target[0],current_target[1]] =1
            maze.terrain_matrix[current_target[0],current_target[1]] = 3
            
            continue
        
        for i in range(len(path)):
            node = path[i]
            agent7.actions += 1
            if full_grid_world_matrix.item(node) == 0:
                agent7.visited_matrix.itemset(node, True)
                terrain_type = terrain_matrix.item(node)
                agent7.agent_gridworld.itemset(node, terrain_type)
                if i == len(path)-1:
                    agent7.actions += 1
                    if(terrain_type == 0):
                        agent7.flat_exam += 1
                    elif(terrain_type == 1):
                        agent7.hilly_exam += 1
                    else:
                        agent7.forest_exam += 1
                    if(agent7.examine(node, terrain_type, maze)):
#                         print("found goal")
                        target_reached = True
                        break
                    else:
                        #update probabiities when failed to be found and terrain type=terrain_matrix.item(node)
                        agent7.update_beliefs_when_xy_failed_with_some_terrainType(node[0], node[1], terrain_type, maze_size)
                        agent7.update_certainties(maze)
                        source = path[i]
#                         print(agent7.belief_matrix)
                        break
            elif full_grid_world_matrix.item(node) == 1:
                agent7.actions += 2
                agent7.agent_gridworld_0_1[node[0],node[1]] = 1
                agent7.agent_gridworld.itemset(node,3)
                agent7.update_beliefs_when_xy_blocked(node[0], node[1], maze_size)
                agent7.update_certainties(maze)
                
                source = path[i-1]
#                 print("after:",agent7.belief_matrix)
                break


In [11]:
class agent_7:

    def __init__(self, maze_size):
        self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
        self.visited_matrix = np.matrix(np.zeros((maze_size, maze_size),dtype=bool))
        self.agent_gridworld = np.matrix(np.zeros((maze_size, maze_size)))
        self.agent_gridworld_0_1 = np.matrix(np.zeros((maze_size, maze_size)))
        self.certainty_matrix = np.full((maze_size, maze_size), 0.35/maze_size*maze_size)
        self.actions = 0
        self.flat_exam = 0
        self.hilly_exam =0
        self.forest_exam = 0
        

#     def display_node(self):
#         print("Node: ", self.row, "," , self.column)
#         print("\n the number of neighbors cell x has: ", self.Nx)
#         print("visited  Whether or not cell x has been visited: ", self.visited)
#         print("blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed: ", self.blocked)
#         print("The undiscovered neighbors are: ", self.neighbor_list)
#         print("The confirmed neighbors that are free: ", self.free_neighbors)
#         print("The confirmed neighbors that are blocked: ", self.blocked_neighbors)

        
       
    def getMaxCertaintyCell(self, source):
        mat = self.certainty_matrix
        
        maxProb = mat.max()
        occurences = list(np.where(mat == maxProb))
        number_of_occurrences = len(occurences[0])
        node_to_return = source
        
        if number_of_occurrences > 1:
            dist = []
            previous_manhattan_distance = 9999
            for index in range(number_of_occurrences):
                
                _row = occurences[0][index]
                _col = occurences[1][index]
                
                target_node = (_row, _col)
                # find manhattan distance
                current_manhattan_distance = cityblock([source], target_node)
                dist.append([target_node,current_manhattan_distance])

                # code for finding manhattan distance between current node and the node in loop
                if current_manhattan_distance < previous_manhattan_distance:
                    previous_manhattan_distance = current_manhattan_distance
            dist = [x for x in dist if x[1]==previous_manhattan_distance]
            node_to_return = dist[np.random.randint(0,len(dist))][0]
        else:
            row = occurences[0][0]
            col = occurences[1][0]
            node_to_return = (row, col)
        
        return node_to_return
    
    def update_certainties(self, maze):
        
        t_mat = maze.terrain_matrix
        bel_mat = self.belief_matrix
        v_mat = self.visited_matrix
        c_mat = self.certainty_matrix
        
        for i in range(maze_size):
            for j in range(maze_size):
                if v_mat.item(i,j):
                    if t_mat.item(i,j) == 0:
                        c_mat[i,j] = bel_mat[i,j]*0.8
                    if t_mat.item(i,j) == 1:
                        c_mat[i,j] = bel_mat[i,j]*0.5
                    if t_mat.item(i,j) == 2:
                        c_mat[i,j] = bel_mat[i,j]*0.2
                else:
                    c_mat[i,j] = bel_mat[i,j]*0.35
    
    
    def examine(self, node, terrain_type, maze):
        if node ==  maze.target:
#             print("examine target")
            p = np.random.rand()
            if terrain_type == 0:
                if p >= 0.2:
                    return True
            elif terrain_type == 1:
                if p >= 0.5:
                    return True
            elif terrain_type == 2:
                if p >= 0.8:
                    return True
        
        return False
        
    def update_beliefs_when_xy_blocked(self, x, y, maze_size):
        denominator = 1 - self.belief_matrix[x][y]
        self.belief_matrix = self.belief_matrix/denominator
        
        self.belief_matrix[x][y]=0
        
        
#         print(self.belief_matrix)
    
    
    def update_beliefs_when_xy_failed_with_some_terrainType(self, x, y, t, maze_size):
        
        if t == 0:
            p_xy = 0.2*self.belief_matrix[x][y]
        elif t == 1:
            p_xy= 0.5*self.belief_matrix[x][y]
        elif t == 2:
            p_xy= 0.8*self.belief_matrix[x][y]
        
        self.belief_matrix[x][y] = p_xy
        self.belief_matrix = self.belief_matrix/self.belief_matrix.sum()
        
        
#         print(self.belief_matrix)

In [12]:
def run_agent_8(agent8, maze):
#     print("Agent6 started.\nMaze Size: ",maze_size," Source: ",maze.source," Target: ",maze.target)
    global maze_size
    source = maze.source
    terrain_matrix = maze.terrain_matrix
    full_grid_world_matrix = maze.full_grid_world_matrix
    agent_gridworld = agent8.agent_gridworld

    target_reached = False
    while not target_reached:
        current_target = agent8.getMaxProbabilityCell(source, maze_size)
#         print("maxProbCell",current_target)
#         print('source:', source, "current_target: ",current_target)
        
#             print("astar args: source:",source,"target:",current_target,"gridworld:",agent6.agent_gridworld_0_1 )
        initialize(maze_size,0.3)
        path = astar(source,current_target, agent8.agent_gridworld_0_1)
#         print(path)\
        if(len(path)==0):
            agent8.agent_gridworld_0_1[current_target[0],current_target[1]] = 1
            agent8.update_cells_when_xy_blocked(current_target[0], current_target[1], maze_size)
            agent8.update_certainties(maze)
            
            agent8.agent_gridworld.itemset(current_target,3)
            maze.full_grid_world_matrix[current_target[0],current_target[1]] =1
            maze.terrain_matrix[current_target[0],current_target[1]] = 3
            continue
        
        for i in range(len(path)):
            node = path[i]
            agent8.actions += 1
            if full_grid_world_matrix.item(node) == 0:
                terrain_type = terrain_matrix.item(node)
                agent8.visited_matrix.itemset(node, True)
                agent8.agent_gridworld.itemset(node, terrain_type)
                if i == len(path)-1:
                    agent8.actions += 1
                    if(terrain_type == 0):
                        agent8.flat_exam += 1
                    elif(terrain_type == 1):
                        agent8.hilly_exam += 1
                    else:
                        agent8.forest_exam += 1
                    if(agent8.examine(node, terrain_type, maze)):
#                         print("found goal")
                        target_reached = True
                        break
                    else:
                        #update probabiities when failed to be found and terrain type=terrain_matrix.item(node)
                        agent8.update_cells_when_xy_failed_with_some_terrainType(node[0], node[1], terrain_type, maze_size)
                        agent8.update_certainties(maze)
                        source = path[i]
#                         print(agent6.belief_matrix)
                        break
            elif full_grid_world_matrix.item(node) == 1:
                agent8.actions += 2
#                 print(node,'<-cell blocked, replanning route')
                #agent6.update_probabilities(node,'blocked')
                agent8.agent_gridworld_0_1[node[0],node[1]] = 1
#                 print("In node:",node,"\nbefore: ",agent6.belief_matrix)
                agent8.update_cells_when_xy_blocked(node[0], node[1], maze_size)
                agent8.update_certainties(maze)
                agent8.agent_gridworld.itemset(node,3)
                source = path[i-1]
#                 print("after:",agent6.belief_matrix)
                break


In [13]:
class agent_8:

    def __init__(self, maze_size):
        self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
        self.visited_matrix = np.matrix(np.zeros((maze_size, maze_size),dtype=bool))
        self.certainty_matrix = np.full((maze_size, maze_size), 0.35/maze_size*maze_size)
        self.agent_gridworld = np.matrix(np.zeros((maze_size, maze_size)))
        self.agent_gridworld_0_1 = np.matrix(np.zeros((maze_size, maze_size)))
        self.actions = 0
        self.flat_exam = 0
        self.hilly_exam =0
        self.forest_exam = 0

#     def display_node(self):
#         print("Node: ", self.row, "," , self.column)
#         print("\n the number of neighbors cell x has: ", self.Nx)
#         print("visited  Whether or not cell x has been visited: ", self.visited)
#         print("blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed: ", self.blocked)
#         print("The undiscovered neighbors are: ", self.neighbor_list)
#         print("The confirmed neighbors that are free: ", self.free_neighbors)
#         print("The confirmed neighbors that are blocked: ", self.blocked_neighbors)

        
#     def getMaxCertaintyCell(self, source):
#         mat = self.certainty_matrix
        
#         maxProb = mat.max()
#         occurences = list(np.where(mat == maxProb))
#         number_of_occurrences = len(occurences[0])
#         node_to_return = source
        
#         if number_of_occurrences > 1:
#             dist = []
#             previous_manhattan_distance = 9999
#             for index in range(number_of_occurrences):
                
#                 _row = occurences[0][index]
#                 _col = occurences[1][index]
                
#                 target_node = (_row, _col)
#                 # find manhattan distance
#                 current_manhattan_distance = cityblock([source], target_node)
#                 dist.append([target_node,current_manhattan_distance])

#                 # code for finding manhattan distance between current node and the node in loop
#                 if current_manhattan_distance < previous_manhattan_distance:
#                     previous_manhattan_distance = current_manhattan_distance
#             dist = [x for x in dist if x[1]==previous_manhattan_distance]
#             node_to_return = dist[np.random.randint(0,len(dist))][0]
#         else:
#             row = occurences[0][0]
#             col = occurences[1][0]
#             node_to_return = (row, col)
        
#         return node_to_return
    
    def update_certainties(self, maze):
        
        t_mat = maze.terrain_matrix
        bel_mat = self.belief_matrix
        v_mat = self.visited_matrix
        c_mat = self.certainty_matrix
        
        for i in range(maze_size):
            for j in range(maze_size):
                if v_mat.item(i,j):
                    if t_mat.item(i,j) == 0:
                        c_mat[i,j] = bel_mat[i,j]*0.8
                    if t_mat.item(i,j) == 1:
                        c_mat[i,j] = bel_mat[i,j]*0.5
                    if t_mat.item(i,j) == 2:
                        c_mat[i,j] = bel_mat[i,j]*0.2
                else:
                    c_mat[i,j] = bel_mat[i,j]*0.35
        self.certainty_matrix = c_mat
    
    def getMaxProbabilityCell(self, source, maze_size):
        #print("belief matrix:", self.belief_matrix)
        mat = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
        for i in range(maze_size):
            for j in range(maze_size):
                manhattan_dist = cityblock([source], (i,j))
                if manhattan_dist!=0:
                    mat[i][j] = self.certainty_matrix[i][j]/manhattan_dist
                else:
                    mat[i][j] = self.certainty_matrix[i][j]
        #print("belief distance matrix:", mat)
        
        maxProb = mat.max()
        occurences = list(np.where(mat == maxProb))
        number_of_occurrences = len(occurences[0])
        node_to_return = source
        if number_of_occurrences > 1:
            dist = []
            previous_manhattan_distance = 9999
            for index in range(number_of_occurrences):
                
                _row = occurences[0][index]
                _col = occurences[1][index]
                
                target_node = (_row, _col)
                # find manhattan distance
                current_manhattan_distance = cityblock([source], target_node)
                dist.append([target_node,current_manhattan_distance])

                # code for finding manhattan distance between current node and the node in loop
                if current_manhattan_distance < previous_manhattan_distance:
                    previous_manhattan_distance = current_manhattan_distance
#                     node_to_return = target_node
            dist = [x for x in dist if x[1]==previous_manhattan_distance]
            node_to_return = dist[np.random.randint(0,len(dist))][0]
        
        else:
            row = occurences[0][0]
            col = occurences[1][0]
            node_to_return = (row, col)
        
        
        return node_to_return
    
    
    def examine(self, node, terrain_type, maze):
        if node ==  maze.target:
#             print("examine target")
            p = np.random.rand()
            if terrain_type == 0:
                if p >= 0.2:
                    return True
            elif terrain_type == 1:
                if p >= 0.5:
                    return True
            elif terrain_type == 2:
                if p >= 0.8:
                    return True
        
        return False
        
    def update_cells_when_xy_blocked(self, x, y, maze_size):
        denominator = 1 - self.belief_matrix[x][y]
        self.belief_matrix = self.belief_matrix/denominator
        
        self.belief_matrix[x][y]=0
        
        
#         print(self.belief_matrix)
    
    
    def update_cells_when_xy_failed_with_some_terrainType(self, x, y, t, maze_size):
        
        if t == 0:
            p_xy = 0.2*self.belief_matrix[x][y]
        elif t == 1:
            p_xy= 0.5*self.belief_matrix[x][y]
        elif t == 2:
            p_xy= 0.8*self.belief_matrix[x][y]
        
        self.belief_matrix[x][y] = p_xy
        self.belief_matrix = self.belief_matrix/self.belief_matrix.sum()

In [14]:
class agent_8_2:

    def __init__(self, maze_size):
        self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
        self.visited_matrix = np.matrix(np.zeros((maze_size, maze_size),dtype=bool))
        self.certainty_matrix = np.full((maze_size, maze_size), 0.35/maze_size*maze_size)
        self.agent_gridworld = np.matrix(np.zeros((maze_size, maze_size)))
        self.agent_gridworld_0_1 = np.matrix(np.zeros((maze_size, maze_size)))
        self.actions = 0
        self.flat_exam = 0
        self.hilly_exam =0
        self.forest_exam = 0

#     def display_node(self):
#         print("Node: ", self.row, "," , self.column)
#         print("\n the number of neighbors cell x has: ", self.Nx)
#         print("visited  Whether or not cell x has been visited: ", self.visited)
#         print("blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed: ", self.blocked)
#         print("The undiscovered neighbors are: ", self.neighbor_list)
#         print("The confirmed neighbors that are free: ", self.free_neighbors)
#         print("The confirmed neighbors that are blocked: ", self.blocked_neighbors)

        
#     def getMaxCertaintyCell(self, source):
#         mat = self.certainty_matrix
        
#         maxProb = mat.max()
#         occurences = list(np.where(mat == maxProb))
#         number_of_occurrences = len(occurences[0])
#         node_to_return = source
        
#         if number_of_occurrences > 1:
#             dist = []
#             previous_manhattan_distance = 9999
#             for index in range(number_of_occurrences):
                
#                 _row = occurences[0][index]
#                 _col = occurences[1][index]
                
#                 target_node = (_row, _col)
#                 # find manhattan distance
#                 current_manhattan_distance = cityblock([source], target_node)
#                 dist.append([target_node,current_manhattan_distance])

#                 # code for finding manhattan distance between current node and the node in loop
#                 if current_manhattan_distance < previous_manhattan_distance:
#                     previous_manhattan_distance = current_manhattan_distance
#             dist = [x for x in dist if x[1]==previous_manhattan_distance]
#             node_to_return = dist[np.random.randint(0,len(dist))][0]
#         else:
#             row = occurences[0][0]
#             col = occurences[1][0]
#             node_to_return = (row, col)
        
#         return node_to_return
    
    def update_certainties(self, maze):
        
        t_mat = maze.terrain_matrix
        bel_mat = self.belief_matrix
        v_mat = self.visited_matrix
        c_mat = self.certainty_matrix
        
        for i in range(maze_size):
            for j in range(maze_size):
                if v_mat.item(i,j):
                    if t_mat.item(i,j) == 0:
                        c_mat[i,j] = bel_mat[i,j]*0.8
                    if t_mat.item(i,j) == 1:
                        c_mat[i,j] = bel_mat[i,j]*0.5
                    if t_mat.item(i,j) == 2:
                        c_mat[i,j] = bel_mat[i,j]*0.2
                else:
                    c_mat[i,j] = bel_mat[i,j]*0.35
        self.certainty_matrix = c_mat
    
    def getMaxProbabilityCellDivideByDistance(self, source, maze_size):
        #print("belief matrix:", self.belief_matrix)
        mat = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
        for i in range(maze_size):
            for j in range(maze_size):
                manhattan_dist = cityblock([source], (i,j))
                if manhattan_dist!=0:
                    mat[i][j] = self.certainty_matrix[i][j]/manhattan_dist
                else:
                    mat[i][j] = self.certainty_matrix[i][j]
        #print("belief distance matrix:", mat)
        
        maxProb = mat.max()
        occurences = list(np.where(mat == maxProb))
        number_of_occurrences = len(occurences[0])
        node_to_return = source
        if number_of_occurrences > 1:
            dist = []
            previous_manhattan_distance = 9999
            for index in range(number_of_occurrences):
                
                _row = occurences[0][index]
                _col = occurences[1][index]
                
                target_node = (_row, _col)
                # find manhattan distance
                current_manhattan_distance = cityblock([source], target_node)
                dist.append([target_node,current_manhattan_distance])

                # code for finding manhattan distance between current node and the node in loop
                if current_manhattan_distance < previous_manhattan_distance:
                    previous_manhattan_distance = current_manhattan_distance
#                     node_to_return = target_node
            dist = [x for x in dist if x[1]==previous_manhattan_distance]
            node_to_return = dist[np.random.randint(0,len(dist))][0]
        
        else:
            row = occurences[0][0]
            col = occurences[1][0]
            node_to_return = (row, col)
        
        
        return node_to_return
    
    
    
    def getMaxCertaintyCell(self, source):
        mat = self.certainty_matrix
        
        maxProb = mat.max()
        occurences = list(np.where(mat == maxProb))
        number_of_occurrences = len(occurences[0])
        node_to_return = source
        
        if number_of_occurrences > 1:
            dist = []
            previous_manhattan_distance = 9999
            for index in range(number_of_occurrences):
                
                _row = occurences[0][index]
                _col = occurences[1][index]
                
                target_node = (_row, _col)
                # find manhattan distance
                current_manhattan_distance = cityblock([source], target_node)
                dist.append([target_node,current_manhattan_distance])

                # code for finding manhattan distance between current node and the node in loop
                if current_manhattan_distance < previous_manhattan_distance:
                    previous_manhattan_distance = current_manhattan_distance
            dist = [x for x in dist if x[1]==previous_manhattan_distance]
            node_to_return = dist[np.random.randint(0,len(dist))][0]
        else:
            row = occurences[0][0]
            col = occurences[1][0]
            node_to_return = (row, col)
        
        return node_to_return
    
    def examine(self, node, terrain_type, maze):
        if node ==  maze.target:
#             print("examine target")
            p = np.random.rand()
            if terrain_type == 0:
                if p >= 0.2:
                    return True
            elif terrain_type == 1:
                if p >= 0.5:
                    return True
            elif terrain_type == 2:
                if p >= 0.8:
                    return True
        
        return False
        
    
    def examine(self, node, terrain_type, maze):
        if node ==  maze.target:
#             print("examine target")
            p = np.random.rand()
            if terrain_type == 0:
                if p >= 0.2:
                    return True
            elif terrain_type == 1:
                if p >= 0.5:
                    return True
            elif terrain_type == 2:
                if p >= 0.8:
                    return True
        
        return False
        
    def update_cells_when_xy_blocked(self, x, y, maze_size):
        denominator = 1 - self.belief_matrix[x][y]
        self.belief_matrix = self.belief_matrix/denominator
        
        self.belief_matrix[x][y]=0
        
        
#         print(self.belief_matrix)
    
    
    def update_cells_when_xy_failed_with_some_terrainType(self, x, y, t, maze_size):
        
        if t == 0:
            p_xy = 0.2*self.belief_matrix[x][y]
        elif t == 1:
            p_xy= 0.5*self.belief_matrix[x][y]
        elif t == 2:
            p_xy= 0.8*self.belief_matrix[x][y]
        
        self.belief_matrix[x][y] = p_xy
        self.belief_matrix = self.belief_matrix/self.belief_matrix.sum()

In [15]:
def run_agent_8_2(agent8, maze):
#     print("Agent6 started.\nMaze Size: ",maze_size," Source: ",maze.source," Target: ",maze.target)
    global maze_size
    source = maze.source
    terrain_matrix = maze.terrain_matrix
    full_grid_world_matrix = maze.full_grid_world_matrix
    agent_gridworld = agent8.agent_gridworld

    current_target_type = 1
    current_target2 = (0,0)
    
    target_reached = False
    
    while not target_reached:
        
        if current_target_type == 1:
            current_target = agent8.getMaxProbabilityCellDivideByDistance(source, maze_size)
            current_target2 = agent8.getMaxCertaintyCell(source)
            current_target_type = 2
        else:
            current_target = current_target2
            current_target_type = 1
        
        initialize(maze_size,0.3)
        path = astar(source,current_target, agent8.agent_gridworld_0_1)
        if(len(path)==0):
            agent8.agent_gridworld_0_1[current_target[0],current_target[1]] = 1
            agent8.update_cells_when_xy_blocked(current_target[0], current_target[1], maze_size)
            agent8.update_certainties(maze)
            
            agent8.agent_gridworld.itemset(current_target,3)
            maze.full_grid_world_matrix[current_target[0],current_target[1]] =1
            maze.terrain_matrix[current_target[0],current_target[1]] = 3
            continue
        
        for i in range(len(path)):
            node = path[i]
            agent8.actions += 1
            if full_grid_world_matrix.item(node) == 0:
                terrain_type = terrain_matrix.item(node)
                agent8.visited_matrix.itemset(node, True)
                agent8.agent_gridworld.itemset(node, terrain_type)
                if i == len(path)-1:
                    agent8.actions += 1
                    if(terrain_type == 0):
                        agent8.flat_exam += 1
                    elif(terrain_type == 1):
                        agent8.hilly_exam += 1
                    else:
                        agent8.forest_exam += 1
                    if(agent8.examine(node, terrain_type, maze)):
                        print("found goal")
                        target_reached = True
                        break
                    else:
                        #update probabiities when failed to be found and terrain type=terrain_matrix.item(node)
                        agent8.update_cells_when_xy_failed_with_some_terrainType(node[0], node[1], terrain_type, maze_size)
                        agent8.update_certainties(maze)
                        source = path[i]
#                         print(agent6.belief_matrix)
                        break
            elif full_grid_world_matrix.item(node) == 1:
                agent8.actions += 2
#                 print(node,'<-cell blocked, replanning route')
                #agent6.update_probabilities(node,'blocked')
                agent8.agent_gridworld_0_1[node[0],node[1]] = 1
#                 print("In node:",node,"\nbefore: ",agent6.belief_matrix)
                agent8.update_cells_when_xy_blocked(node[0], node[1], maze_size)
                agent8.update_certainties(maze)
                agent8.agent_gridworld.itemset(node,3)
                source = path[i-1]
#                 print("after:",agent6.belief_matrix)
                break


In [16]:
class gridworld:
    
    def __init__(self, maze_size):
#         self.full_grid_world_matrix = []
        self.terrain_matrix = self.initialize_terrain_matrix(maze_size)
        self.full_grid_world_matrix = self.initialize_grid_world(maze_size)
        self.source, self.target = self.set_valid_source_target(maze_size)

    def initialize_terrain_matrix(self, maze_size):
        # generates number between 0-2
        np_array = np_array = np.random.rand(maze_size*maze_size)
        block = lambda x: 3 if x<0.3 else 0 # setting blocks
        vectorized_block = np.vectorize(block)
        np_array = vectorized_block(np_array)
#         print(np_array.reshape(maze_size,maze_size))
        self.full_grid_world_matrix = np_array  #setting full_gridworld_matrix
        
        terrain_probs = lambda x: 3 if x==3 else np.random.rand()
        vectorized_terrain_probs = np.vectorize(terrain_probs)
        np_array = vectorized_terrain_probs(np_array)
#         print(np_array.reshape(maze_size, maze_size))
        
        set_terrain = lambda x: x if x==3 else(0 if x<=0.33 else (1 if x<=0.66 else 2))
        vectorized_set_terrain = np.vectorize(set_terrain)
        np_array = vectorized_set_terrain(np_array)
#         print(np_array.reshape(maze_size, maze_size))
        
        terrain_matrix = np.asmatrix(np_array).reshape(maze_size,maze_size)
#         print(terrain_matrix)
        return terrain_matrix

    def initialize_grid_world(self, maze_size):

        block = lambda x: 1 if x==3 else x
        vectorize_block = np.vectorize(block)
        self.full_grid_world_matrix = vectorize_block(self.full_grid_world_matrix)
        self.full_grid_world_matrix = np.asmatrix(self.full_grid_world_matrix).reshape(maze_size,maze_size)
    #     plt.spy(full_grid_world_matrix)
        return self.full_grid_world_matrix

    def set_valid_source_target(self, maze_size):

        target_row = random.randint(0, maze_size-1)
        target_col = random.randint(0, maze_size-1)

        source_row = random.randint(0, maze_size-1)
        source_col = random.randint(0, maze_size-1)

        while (self.full_grid_world_matrix.item(target_row, target_col) == 1):
            target_row = random.randint(0, maze_size-1)
            target_col = random.randint(0, maze_size-1)
            
        while(self.full_grid_world_matrix.item(source_row, source_col) == 1):
            source_row = random.randint(0, maze_size-1)
            source_col = random.randint(0, maze_size-1)
        
        source = (source_row, source_col)
        target = (target_row, target_col)

        return source, target
               

In [17]:
def main(size):
    global maze_size
    #Initialize Maze
    maze_size = size
    maze = gridworld(maze_size)
    
    maze_6 = copy.deepcopy(maze)
    maze_7 = copy.deepcopy(maze)
    maze_8 = copy.deepcopy(maze)
    
    # set source and target
    initialize(maze_size,0.3)
#     print(maze.source,maze.target, maze.full_grid_world_matrix)
    path = astar(maze.source, maze.target, maze.full_grid_world_matrix)
    while(len(path)==0):
#         print("finding valid source and target")
        maze.source, maze.target = maze.set_valid_source_target(maze_size)
        initialize(maze_size,0.3)
        path = astar(maze.source, maze.target, maze.full_grid_world_matrix)
    
    maze_6.source = maze_7.source = maze_8.source = maze.source
    maze_6.target = maze_7.target = maze_8.target = maze.target
    
    # Initialize Agent 6
    agent6 = agent_6(maze_size)
    agent7 = agent_7(maze_size)
    agent8 = agent_8(maze_size)
    
    #Print all details
#     print("Maze size:",maze_size,"\nSource: ",maze.source,"\nTarget:",maze.target)
#     print("maze 6 source, target: ",maze_6.source,",",maze_6.target,"maze 7 source, target: ",maze_7.source,",",maze_7.target,\
#           " maze 8 source target: ", maze_8.source,",",maze_8.target)
#     display2(maze.terrain_matrix)
    #runnning Agents on maze
    run_agent_6(agent6, maze_6)
    run_agent_7(agent7, maze_7)
    run_agent_8(agent8, maze_8)
    
    return (agent6.actions,agent6.flat_exam, agent6.hilly_exam, agent6.forest_exam,agent7.actions,agent7.flat_exam, agent7.hilly_exam, agent7.forest_exam,\
        agent8.actions,agent8.flat_exam, agent8.hilly_exam, agent8.forest_exam)
    

In [18]:
def generate_actions_data(size,num_of_runs):
    f = open("./results/results_"+str(random.randint(1,1000000))+".csv", "a")
    f.write("agent6.actions,agent6.flat_exam, agent6.hilly_exam,agent6.forest_exam,agent7.actions,agent7.flat_exam,agent7.hilly_exam,agent7.forest_exam,\
            agent8.actions,agent8.flat_exam, agent8.hilly_exam, agent8.forest_exam\n")
    for i in range(num_of_runs):
        print("iteration: ",i)
        values = main(size)
        f.write(str(values)[1:-1]+"\n")
    f.close()
    print("results in: ",f.name)

In [19]:
# generate_actions_data(50,50)

In [20]:
# generate_actions_data(50,50)

In [30]:
def data_for_agent_8_2(size):
    global maze_size
    #Initialize Maze
    maze_size = size
    maze = gridworld(maze_size)
    
    
    # set source and target
    initialize(maze_size,0.3)
#     print(maze.source,maze.target, maze.full_grid_world_matrix)
    path = astar(maze.source, maze.target, maze.full_grid_world_matrix)
    while(len(path)==0):
#         print("finding valid source and target")
        maze.source, maze.target = maze.set_valid_source_target(maze_size)
        initialize(maze_size,0.3)
        path = astar(maze.source, maze.target, maze.full_grid_world_matrix)
    
    maze.source = maze.source
    maze.target = maze.target
    
    # Initialize Agent 6
    agent8 = agent_8_2(maze_size)
    
    #Print all details
#     print("Maze size:",maze_size,"\nSource: ",maze.source,"\nTarget:",maze.target)
#     print("maze 6 source, target: ",maze_6.source,",",maze_6.target,"maze 7 source, target: ",maze_7.source,",",maze_7.target,\
#           " maze 8 source target: ", maze_8.source,",",maze_8.target)
#     display2(maze.terrain_matrix)
    #runnning Agents on maze
#     run_agent_6(agent6, maze_6)
#     run_agent_7(agent7, maze_7)
    run_agent_8_2(agent8, maze)
    
    return (agent8.actions,agent8.flat_exam, agent8.hilly_exam, agent8.forest_exam)
    

In [33]:
def generate_actions_data_agent_8_2(size,num_of_runs):
    f = open("./results/results_"+str(random.randint(1,1000000))+".csv", "a")
    f.write("agent8_2.actions,agent8_2.flat_exam, agent8_2.hilly_exam, agent8_2.forest_exam\n")
    for i in range(num_of_runs):
        print("iteration: ",i)
        values = data_for_agent_8_2(size)
        f.write(str(values)[1:-1]+"\n")
    f.close()
    print("results in: ",f.name)

In [34]:
generate_actions_data_agent_8_2(50,50)

iteration:  0
found goal
iteration:  1
found goal
iteration:  2
found goal
iteration:  3
found goal
iteration:  4
found goal
iteration:  5
found goal
iteration:  6
found goal
iteration:  7
found goal
iteration:  8
found goal
iteration:  9
found goal
iteration:  10
found goal
iteration:  11
found goal
iteration:  12
found goal
iteration:  13
found goal
iteration:  14
found goal
iteration:  15
found goal
iteration:  16
found goal
iteration:  17
found goal
iteration:  18
found goal
iteration:  19
found goal
iteration:  20
found goal
iteration:  21
found goal
iteration:  22
found goal
iteration:  23
found goal
iteration:  24
found goal
iteration:  25
found goal
iteration:  26
found goal
iteration:  27
found goal
iteration:  28
found goal
iteration:  29
found goal
iteration:  30
found goal
iteration:  31
found goal
iteration:  32
found goal
iteration:  33
found goal
iteration:  34
found goal
iteration:  35
found goal
iteration:  36
found goal
iteration:  37
found goal
iteration:  38
found g

In [35]:
generate_actions_data_agent_8_2(50,50)

iteration:  0
found goal
iteration:  1
found goal
iteration:  2
found goal
iteration:  3
found goal
iteration:  4
found goal
iteration:  5
found goal
iteration:  6
found goal
iteration:  7
found goal
iteration:  8
found goal
iteration:  9
found goal
iteration:  10
found goal
iteration:  11
found goal
iteration:  12
found goal
iteration:  13
found goal
iteration:  14
found goal
iteration:  15
found goal
iteration:  16
found goal
iteration:  17
found goal
iteration:  18
found goal
iteration:  19
found goal
iteration:  20
found goal
iteration:  21
found goal
iteration:  22
found goal
iteration:  23
found goal
iteration:  24
found goal
iteration:  25
found goal
iteration:  26
found goal
iteration:  27
found goal
iteration:  28
found goal
iteration:  29
found goal
iteration:  30
found goal
iteration:  31
found goal
iteration:  32
found goal
iteration:  33
found goal
iteration:  34
found goal
iteration:  35
found goal
iteration:  36
found goal
iteration:  37
found goal
iteration:  38
found g